In [ ]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians

In [ ]:
@load "spielmanIPM_all.jl.split02merge02.jld2"
dic1 = copy(dic)

In [ ]:
import Laplacians.initDictCol!
dics = [Dict(),Dict(),Dict(),Dict(),Dict(),Dict()]

for key in keys(dic1)
    for i in 1:length(dics)
        initDictCol!(dics[i], key, typeof(dic1[key][1]))
        #@show dics[i]
    end
end


for i in 1:length(dic1["testName"])
    tn = dic1["testName"][i]
    curK = parse(Int64, (split(tn, ".")[2][2:end]))
    kInd = round(Int64, curK / 100)
    for key in keys(dic1)
        if key == "names"
            continue
        end
        push!(dics[kInd][key], dic1[key][i])
    end
end

for i in 1:length(dics)
    dics[i]["names"] = dic1["names"]
end

In [ ]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

In [ ]:
nanErr!.(dics);

In [ ]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe3(s) = (@sprintf "%.3g" sd(3,s)*1e3)

In [ ]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

In [ ]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffK(d["ne"][1]),"&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for i in 1:length(dics)
        printOneLine(dics[i], solvers, i)
    end
end

In [ ]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

In [ ]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)